<a href="https://colab.research.google.com/github/GUNAPILLCO/neural_profit/blob/main/2_obtencion_preparacion_exploracion_datos/2_3_indicadores_tecnicos_retorno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2_3_Análisis de indicadores técnicos




## 0. Clonado de repositorio, importación de librerías y carga del dataset

### Clonado de repositorio e importación de librerías

In [1]:
#Clonamos el repo
#LINK DE REPOSITORIO: https://github.com/GUNAPILLCO/neural_profit

!git clone https://github.com/GUNAPILLCO/neural_profit.git

Cloning into 'neural_profit'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 186 (delta 64), reused 29 (delta 17), pack-reused 89 (from 1)
Receiving objects: 100% (186/186), 108.59 MiB | 25.28 MiB/s, done.
Resolving deltas: 100% (97/97), done.
Updating files: 100% (41/41), done.


In [2]:
import sys

#Instalación de librería pandas_market_calendars
#!{sys.executable} -m pip install -q pandas_market_calendars
#print("Librería instalada: pandas_market_calendars")

!{sys.executable} -m pip install -q ta
print("Librería instalada: technical-analysis")

# Utilidades generales
from datetime import datetime, timedelta
import os
import glob
import requests
import warnings
warnings.filterwarnings('ignore')

# Manejo y procesamiento de datos
import ta
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
# Calendario de mercados
#import pandas_market_calendars as mcal

from ta.momentum import StochasticOscillator, ROCIndicator
from ta.volatility import BollingerBands, AverageTrueRange

from scipy.stats import spearmanr


  Preparing metadata (setup.py) ... done
Librería instalada: technical-analysis


### Carga del dataset mnq_intraday_data

In [3]:
def load_df():
    """
    Función para cargar un archivo Parquet desde el repositorio clonado
    """
    # Definir la URL del archivo Parquet en GitHub
    df_path = '/content/neural_profit/2_obtencion_preparacion_exploracion_datos/mnq_intraday_data.parquet'

    # Leer el archivo Parquet y cargarlo en un DataFrame
    df = pd.read_parquet(df_path)

    # Asegurar que el índice esté en formato datetime
    df.index = pd.to_datetime(df.index)

    # Crear una nueva columna 'date' con la fecha extraída del índice
    df['date'] = df.index.date

    # Reordenar columnas: 'date', 'time_str', y luego el resto
    cols = ['date'] + [col for col in df.columns if col not in ['date']]

    df = df[cols]

    return df

In [4]:
mnq_intraday = load_df()

## 1. Definición de Target

Se utiliza el retorno futuro a 30 minutos (target_return_30) como variable objetivo del modelo porque, tras evaluar múltiples hipótesis de predicción, fue el horizonte temporal con mayor IC medio para los indicadores técnicos seleccionados.

Por lo tanto, usaremos los indicadores técnicos para analizar el comportamiento de dicha variable objetivo.



In [5]:
def add_targets (df):
  df['target_return_30'] = df.groupby('date')['close'].transform(lambda x: np.log(x.shift(-30)) - np.log(x))
  return df

In [6]:
mnq_intraday = add_targets(mnq_intraday)

In [7]:
mnq_intraday

,date,open,high,low,close,volume,target_return_30
datetime,,,,,,,
2019-12-23 08:00:00-05:00,2019-12-23,8734.00,8734.25,8733.75,8733.75,31,0.000372
2019-12-23 08:01:00-05:00,2019-12-23,8734.00,8734.25,8733.75,8734.00,16,0.000286
2019-12-23 08:02:00-05:00,2019-12-23,8734.00,8734.00,8733.25,8733.25,23,0.000315
2019-12-23 08:03:00-05:00,2019-12-23,8734.25,8734.50,8734.00,8734.00,23,0.000315
2019-12-23 08:04:00-05:00,2019-12-23,8734.00,8734.00,8733.50,8733.75,12,0.000458
...,...,...,...,...,...,...,...
2025-06-13 15:56:00-04:00,2025-06-13,21624.50,21635.00,21613.50,21617.50,3251,NaN
2025-06-13 15:57:00-04:00,2025-06-13,21616.50,21635.25,21615.75,21623.75,2201,NaN
2025-06-13 15:58:00-04:00,2025-06-13,21623.25,21632.75,21616.50,21621.75,1859,NaN


## 2. Indicadores Técnicos

Los indicadores técnicos calculados en cada jornada tienen como objetivo capturar dinámicas intradía relevantes del precio y el volumen, tales como momentum, sobrecompra/sobreventa, presión institucional o posibles reversiones. Cada uno aporta información complementaria sobre el comportamiento del mercado a corto plazo. En particular:

### 2.1. **RSI (Relative Strength Index)**

Mide la fuerza relativa del precio en los últimos períodos (3, 5, 7, 14), oscilando entre 0 y 100.

  - Valores altos indican posibles condiciones de sobrecompra, mientras que valores bajos sugieren sobreventa.
  
  - Calculado sobre los precios de cierre intradía, el RSI es útil para identificar puntos de reversión potenciales en el corto plazo.

In [8]:
def calcular_rsi(df=mnq_intraday, target='target_return_30' ):
  rsi_columns = ['rsi_14', 'rsi_7', 'rsi_5', 'rsi_3']

  def aplicar_por_dia (grupo):
        grupo = grupo.copy()
        grupo['rsi_14'] = ta.momentum.RSIIndicator(grupo[target], window=14).rsi()
        grupo['rsi_7'] = ta.momentum.RSIIndicator(grupo[target], window=7).rsi()
        grupo['rsi_5'] = ta.momentum.RSIIndicator(grupo[target], window=5).rsi()
        grupo['rsi_3'] = ta.momentum.RSIIndicator(grupo[target], window=3).rsi()
        return grupo

  df = df.groupby('date', group_keys=False).apply(aplicar_por_dia)
  return df, rsi_columns

### 2.2. **Momentum**

Mide la aceleración reciente del precio mediante la variación porcentual entre el precio actual y el de hace N minutos.

  - Un valor positivo indica una subida reciente, lo que podría sugerir una continuación alcista.

  - Un valor negativo señala presión bajista reciente, potencialmente anticipando una continuación a la baja.

In [9]:
def calcular_momentum(df=mnq_intraday, target='target_return_30' ):
  momentum_columns = ['momentum_10', 'momentum_5','momentum_3']

  def aplicar_por_dia (grupo):
        grupo = grupo.copy()
        grupo['momentum_10'] = grupo[target].pct_change(10)
        grupo['momentum_5'] = grupo[target].pct_change(5)
        grupo['momentum_3'] = grupo[target].pct_change(3)
        return grupo

  df = df.groupby('date', group_keys=False).apply(aplicar_por_dia)
  return df, momentum_columns

### 2.3. **Relación de volumen actual vs. su promedio reciente**

Compara el volumen actual con su media móvil en distintas ventanas de tiempo: 15, 20 y 30 minutos.

  - Un valor mayor a 1 indica un volumen superior al promedio de la ventana correspondiente, lo que puede reflejar interés creciente o actividad institucional.

  - Un valor menor a 1 sugiere baja actividad o consolidación del precio.

Esta métrica permite detectar aumentos de volumen ("spikes") sin depender del volumen en crudo, y las diferentes ventanas permiten capturar variaciones en la dinámica de corto plazo con distinta sensibilidad.


In [10]:
def calcular_volumen_ratio(df=mnq_intraday, target='target_return_30'):
  volume_ratio_columns = ['volume_ratio_15', 'volume_ratio_20', 'volume_ratio_30']

  def aplicar_por_dia (grupo):
        grupo = grupo.copy()
        grupo['volume_ratio_15'] = grupo['volume'] / grupo['volume'].rolling(15).mean()
        grupo['volume_ratio_20'] = grupo['volume'] / grupo['volume'].rolling(20).mean()
        grupo['volume_ratio_30'] = grupo['volume'] / grupo['volume'].rolling(30).mean()
        return grupo

  df = df.groupby('date', group_keys=False).apply(aplicar_por_dia)

  return df, volume_ratio_columns

### 2.4. **MACD diferencial (señal de cruce)**

Representa la diferencia entre la línea MACD y su línea de señal (una media exponencial de sí misma).

  - Un valor positivo y creciente indica momentum alcista.

  - Un valor negativo sugiere presión bajista.
  
Es ampliamente utilizado para detectar giros de tendencia y cambios en la dinámica del mercado.


In [11]:
def calcular_macd(df=mnq_intraday, target='target_return_30'):
    macd_columns = ['macd']

    def aplicar_por_dia(grupo):
        grupo = grupo.copy()
        grupo['macd'] = ta.trend.MACD(grupo[target]).macd_diff()
        return grupo

    df = df.groupby('date', group_keys=False).apply(aplicar_por_dia)

    return df, macd_columns

### 2.5. **Distancia del precio actual a su EMA (15, 20 y 30 minutos)**

Mide el desvío porcentual del precio respecto a su media exponencial en diferentes ventanas, y actúa como indicador de sobreextensión o retorno a la media.

  - Si el precio está muy por encima de la EMA, puede anticipar una reversión bajista o una posible aceleración alcista.

  - Si está por debajo, podría indicar agotamiento o presión vendedora.<br>

Esta métrica se expresa como un porcentaje relativo, lo que facilita la comparación entre distintas ventanas temporales y condiciones de mercado.

Usar varias ventanas (15, 20 y 30 minutos) permite capturar diferentes horizontes de reacción del precio frente a su media móvil.


In [12]:
def calcular_ema(df=mnq_intraday, target='target_return_30'):
    ema_columns = ['price_ema15', 'price_ema20', 'price_ema30']

    def aplicar_por_dia(grupo):
        grupo = grupo.copy()
        grupo['price_ema15'] = grupo[target] / grupo[target].ewm(span=15).mean() - 1
        grupo['price_ema20'] = grupo[target] / grupo[target].ewm(span=20).mean() - 1
        grupo['price_ema30'] = grupo[target] / grupo[target].ewm(span=30).mean() - 1
        return grupo

    df = df.groupby('date', group_keys=False).apply(aplicar_por_dia)

    return df, ema_columns

### 2.6. **%K Estocástico**

Mide la posición relativa del precio actual dentro del rango alto-bajo de los últimos n periodos (generalmente 14).

  - Se utiliza para identificar condiciones extremas de sobrecompra o sobreventa.

  - Un valor cercano a 100 indica que el precio está cerca del máximo reciente (potencial sobrecompra), mientras que un valor cercano a 0 indica proximidad al mínimo reciente (posible sobreventa).

Es útil para detectar momentos en los que el precio puede estar excesivamente extendido y susceptible a una reversión.


In [13]:
def calcular_stochastic(df=mnq_intraday, target='target_return_30'):
    stoch_columns = ['stoch_k_14', 'stoch_k_20', 'stoch_k_30']

    def aplicar_por_dia(grupo):
        grupo = grupo.copy()

        stoch_14 = StochasticOscillator(
            high=grupo['high'], low=grupo['low'], close=grupo[target], window=14, smooth_window=3
        )
        grupo['stoch_k_14'] = stoch_14.stoch()

        stoch_20 = StochasticOscillator(
            high=grupo['high'], low=grupo['low'], close=grupo[target], window=20, smooth_window=3
        )
        grupo['stoch_k_20'] = stoch_20.stoch()

        stoch_30 = StochasticOscillator(
            high=grupo['high'], low=grupo['low'], close=grupo[target], window=30, smooth_window=3
        )
        grupo['stoch_k_30'] = stoch_30.stoch()

        return grupo

    df = df.groupby('date', group_keys=False).apply(aplicar_por_dia)

    return df, stoch_columns


### 2.7.**%B de Bollinger (Bollinger Band Percent)**

Indica la posición del precio actual en relación con las bandas de Bollinger, que están construidas alrededor de una media móvil usando desviaciones estándar.

  - Un valor de %B > 1 sugiere que el precio está por encima de la banda superior, lo que podría implicar exceso de optimismo o momentum fuerte.

  - Un valor < 0 indica que está por debajo de la banda inferior, posible señal de pánico o sobreventa extrema.

Este indicador es eficaz para identificar zonas de congestión, breakout o reversiones basadas en la volatilidad reciente.


In [14]:
from ta.volatility import BollingerBands

def calcular_bollinger(df=mnq_intraday, target='target_return_30'):
    bollinger_columns = [
        'bb_percent_15_15', 'bb_percent_20_15', 'bb_percent_30_15',
        'bb_percent_15_20', 'bb_percent_20_20', 'bb_percent_30_20',
        'bb_percent_15_25', 'bb_percent_20_25', 'bb_percent_30_25',
    ]

    def aplicar_por_dia(grupo):
        grupo = grupo.copy()

        # std: 1.5
        grupo['bb_percent_15_15'] = BollingerBands(grupo[target], window=15, window_dev=1.5).bollinger_pband()
        grupo['bb_percent_20_15'] = BollingerBands(grupo[target], window=20, window_dev=1.5).bollinger_pband()
        grupo['bb_percent_30_15'] = BollingerBands(grupo[target], window=30, window_dev=1.5).bollinger_pband()

        # std: 2
        grupo['bb_percent_15_20'] = BollingerBands(grupo[target], window=15, window_dev=2).bollinger_pband()
        grupo['bb_percent_20_20'] = BollingerBands(grupo[target], window=20, window_dev=2).bollinger_pband()
        grupo['bb_percent_30_20'] = BollingerBands(grupo[target], window=30, window_dev=2).bollinger_pband()

        # std: 2.5
        grupo['bb_percent_15_25'] = BollingerBands(grupo[target], window=15, window_dev=2.5).bollinger_pband()
        grupo['bb_percent_20_25'] = BollingerBands(grupo[target], window=20, window_dev=2.5).bollinger_pband()
        grupo['bb_percent_30_25'] = BollingerBands(grupo[target], window=30, window_dev=2.5).bollinger_pband()

        return grupo

    df = df.groupby('date', group_keys=False).apply(aplicar_por_dia)

    return df, bollinger_columns


### 2.8. **ATR normalizado (Average True Range / precio)**

Representa la volatilidad absoluta reciente ajustada al nivel del precio.

  - El ATR mide el rango promedio de oscilación de un activo en los últimos n periodos, capturando tanto movimientos bruscos como gaps.

  - Al normalizarlo dividiéndolo por el precio, se obtiene una medida relativa, comparable entre distintos niveles de mercado.

Este indicador es útil para detectar momentos de alta o baja volatilidad intradía, que pueden influir en la confiabilidad de otras señales técnicas.


In [15]:
def calcular_atr(df=mnq_intraday, target='target_return_30'):
    atr_columns = ['atr_norm']

    def aplicar_por_dia(grupo):
        grupo = grupo.copy()
        atr = AverageTrueRange(high=grupo['high'], low=grupo['low'], close=grupo[target], window=14)
        grupo['atr'] = atr.average_true_range()
        grupo['atr_norm'] = grupo['atr'] / grupo[target]
        return grupo

    df = df.groupby('date', group_keys=False).apply(aplicar_por_dia)

    return df, atr_columns


### 2.9. **ROC (Rate of Change)**

Calcula la tasa de cambio porcentual del precio con respecto a su valor n minutos atrás.

- Es un indicador de momentum que capta aceleraciones o desaceleraciones recientes del precio.

- Valores positivos indican presión alcista; negativos, presión bajista.

A diferencia del momentum tradicional, el ROC expresa el cambio de forma normalizada y en porcentaje, lo que facilita su interpretación comparativa entre distintos activos o marcos temporales.


In [16]:
def calcular_roc(df=mnq_intraday, target='target_return_30'):
    roc_columns = ['roc_5']

    def aplicar_por_dia(grupo):
        grupo = grupo.copy()
        grupo['roc_5'] = ROCIndicator(close=grupo[target], window=5).roc()
        return grupo

    df = df.groupby('date', group_keys=False).apply(aplicar_por_dia)

    return df, roc_columns

##3. Calculo de Information Coefficient (IC)

El Information Coefficient (IC) es la correlación de Spearman entre el factor y el retorno futuro. Mide si el ranking del factor predice el ranking del retorno.

- IC > 0: el factor tiene relación positiva → valores altos del factor tienden a preceder retornos positivos.

- IC < 0: relación inversa → valores altos del factor tienden a preceder retornos negativos (reversión).

- IC ≈ 0: el factor es probablemente ruido o no tiene señal útil.

Valores comunes en finanzas:

- Entre 0.05 a 0.1: débil pero útil.

- Mayor a 0.1: muy raro, pero fuerte.

- Menor 0.05 o cercano a 0: probablemente ruido.

### **Función para calcular IC:**

In [37]:
def calcular_IC(df: pd.DataFrame, target: str, factors: list[str]):
    grouped = df.groupby('date')

    print(f"IC promedio por factor (diario) sobre el {target}:\n")

    resultados = []

    for factor in factors:
        daily_ics = []

        for date, group in grouped:
            df_valid = group[[factor, target]].dropna()
            if len(df_valid) >= 20:
                ic = spearmanr(df_valid[factor], df_valid[target]).correlation
                if np.isfinite(ic):
                    daily_ics.append(ic)

        if daily_ics:
            mean_ic = np.mean(daily_ics)
            std_ic = np.std(daily_ics)
            dias_validos = len(daily_ics)
            print(f"{factor:15}: IC medio = {mean_ic:.4f} | std = {std_ic:.4f} | días válidos = {dias_validos}")
            resultados.append({
                'factor': factor,
                'ic_mean': mean_ic,
                'ic_std': std_ic,
                'dias_validos': dias_validos
            })
        else:
            print(f"{factor:15}: sin días válidos para cálculo")

    return pd.DataFrame(resultados)

### 3.1. **RSI (Relative Strength Index)**

In [27]:
mnq_rsi, rsi_cols = calcular_rsi()
IC_rsi = calcular_IC(mnq_rsi, 'target_return_30', rsi_cols)

📊 IC promedio por factor (diario) sobre el target_return_30:

rsi_14         : IC medio = 0.6387 | std = 0.0666 | días válidos = 1311
rsi_7          : IC medio = 0.4740 | std = 0.0582 | días válidos = 1311
rsi_5          : IC medio = 0.3995 | std = 0.0539 | días válidos = 1311
rsi_3          : IC medio = 0.3040 | std = 0.0471 | días válidos = 1311


#### 3.1.1. Conclusiones

| Indicador | IC medio | Desv. Estándar | Comentario |
|-----------|----------|----------------|------------|
| `rsi_14`  | 0.6387   | 0.0666         | Muy fuerte y consistente. |
| `rsi_7`   | 0.4740   | 0.0582         | Robusto, aunque menor que `rsi_14`. |
| `rsi_5`   | 0.3995   | 0.0539         | Señal moderada. |
| `rsi_3`   | 0.3040   | 0.0471         | Menor poder predictivo. |

<br>

- El IC promedio decrece a medida que la ventana del RSI se acorta, lo que sugiere que las señales más estables y suavizadas (como `rsi_14`) se correlacionan mejor con el comportamiento del precio a 30 minutos.

- `rsi_14` destaca claramente como el más informativo, no solo por su IC medio alto (0.6387), sino también por su baja desviación estándar (0.0666), lo que indica consistencia en distintos días.

- Los RSI con ventanas más cortas (`rsi_3`, `rsi_5`, `rsi_7`) muestran un poder predictivo decreciente. Aunque pueden aportar información, son menos efectivos para este horizonte de predicción.


#### 3.1.2. Recomendaciones

- Priorizar `rsi_14` como alpha factor principal para el modelo de predicción a 30 minutos.

- Considerar también `rsi_7` como alternativa o complemento.

- No es necesario incluir todos los RSI simultáneamente. Para evitar multicolinealidad o sobreajuste, se recomienda aplicar técnicas de selección de variables (como regularización o análisis de redundancia).

### 3.2. **Momentum**

In [29]:
mnq_momentum, momentum_cols = calcular_momentum()
IC_momentum = calcular_IC(mnq_momentum, 'target_return_30', momentum_cols)

📊 IC promedio por factor (diario) sobre el target_return_30:

momentum_10    : IC medio = 0.0334 | std = 0.1975 | días válidos = 1311
momentum_5     : IC medio = 0.0315 | std = 0.1653 | días válidos = 1311
momentum_3     : IC medio = 0.0290 | std = 0.1405 | días válidos = 1311


#### 3.2.1. Conclusiones

| Indicador     | IC medio | Desv. Estándar | Comentario                        |
|---------------|----------|----------------|-----------------------------------|
| `momentum_10` | 0.0334   | 0.1975         | Muy bajo poder predictivo, ruidoso. |
| `momentum_5`  | 0.0315   | 0.1653         | Similar a `momentum_10`, poca utilidad. |
| `momentum_3`  | 0.0290   | 0.1405         | Levemente más estable, pero irrelevante. |

<br>

- Todos los indicadores de momentum muestran **IC promedio cercano a cero**, lo que indica **bajo poder predictivo** sobre el retorno a 30 minutos.

- Además, presentan **desviaciones estándar elevadas**, reflejando **alta inestabilidad diaria** en su comportamiento.

- No se observa mejora al cambiar la ventana (3, 5 o 10 minutos), lo que sugiere que este indicador técnico no capta dinámicas relevantes para el horizonte temporal evaluado.




#### 3.2.2. Recomendaciones

- **No se recomienda incluir los indicadores técnicos `momentum_3`, `momentum_5` ni `momentum_10`** como alpha factors en el modelo.

- Su bajo IC y alta varianza indican que **aportan más ruido que señal** en la predicción del retorno a 30 minutos.

- Es preferible priorizar otros indicadores con mayor estabilidad y poder explicativo.

### 3.3. **Relación de volumen actual vs. su promedio reciente**

In [30]:
mnq_volumen, volumen_cols = calcular_volumen_ratio()
IC_volumen = calcular_IC(mnq_volumen, 'target_return_30', volumen_cols)

📊 IC promedio por factor (diario) sobre el target_return_30:

volume_ratio_15: IC medio = 0.0053 | std = 0.0643 | días válidos = 1311
volume_ratio_20: IC medio = 0.0048 | std = 0.0785 | días válidos = 1311
volume_ratio_30: IC medio = 0.0051 | std = 0.1039 | días válidos = 1311


#### 3.3.1. Conclusiones

| Indicador         | IC medio | Desv. Estándar | Comentario                              |
|-------------------|----------|----------------|------------------------------------------|
| `volume_ratio_15` | 0.0053   | 0.0643         | Extremadamente bajo, sin relevancia.     |
| `volume_ratio_20` | 0.0048   | 0.0785         | Similar resultado, algo más inestable.   |
| `volume_ratio_30` | 0.0051   | 0.1039         | Ligeramente más volátil, igual de débil. |

<br>

- Todos los indicadores `volume_ratio` presentan un **IC medio cercano a cero**, lo cual indica que **no tienen capacidad predictiva significativa** sobre el retorno a 30 minutos.

- A pesar de su baja desviación estándar (en especial en `volume_ratio_15`), los valores de IC son **demasiado bajos como para ser considerados útiles**.

- Aumentar la ventana de promedio (15 → 30) no mejora el desempeño, sino que incluso **incrementa la varianza**.


#### 3.3.2. Recomendaciones

- **No se recomienda utilizar los indicadores `volume_ratio_15`, `volume_ratio_20` ni `volume_ratio_30`** como alpha factors.

- Su incapacidad para capturar una señal útil para el horizonte temporal evaluado los vuelve **prescindibles en el modelo**.

- En su lugar, conviene enfocarse en factores de precio o volatilidad con mejor desempeño estadístico.


### 3.4. **MACD diferencial (señal de cruce)**

In [31]:
mnq_macd, macd_cols = calcular_macd()
IC_macd = calcular_IC(mnq_macd, 'target_return_30', macd_cols)

📊 IC promedio por factor (diario) sobre el target_return_30:

macd           : IC medio = 0.1962 | std = 0.0984 | días válidos = 1311


### 3.4.1. Conclusiones

| Indicador | IC medio | Desv. Estándar | Comentario                            |
|-----------|----------|----------------|----------------------------------------|
| `macd`    | 0.1962   | 0.0984         | Señal moderada, con estabilidad aceptable. |

<br>

- El indicador `macd` muestra un **IC medio positivo (0.1962)**, lo cual sugiere que tiene **cierta capacidad predictiva** sobre el retorno a 30 minutos.

- Su desviación estándar (0.0984) es **moderada**, lo que indica un nivel de **consistencia razonable** a lo largo de los días.

- Aunque no alcanza la fuerza predictiva de otros factores como `rsi_14` o `bb_percent_30_20`, el MACD ofrece una **señal útil y complementaria**, especialmente en entornos de tendencia.


#### 3.4.2. Recomendaciones

- Se puede **incluir `macd` como alpha factor secundario**, aportando valor en combinación con otros indicadores más fuertes.

- Su uso es particularmente relevante si el modelo incorpora señales de momentum o detección de cambios de tendencia.

- Es aconsejable validarlo dentro de un conjunto de features para evaluar su contribución marginal.

### 3.5. **Distancia del precio actual a su EMA (15, 20 y 30 minutos)**

In [32]:
mnq_ema, ema_cols = calcular_ema()
IC_ema = calcular_IC(mnq_ema, 'target_return_30', ema_cols)

📊 IC promedio por factor (diario) sobre el target_return_30:

price_ema15    : IC medio = 0.0207 | std = 0.1361 | días válidos = 1311
price_ema20    : IC medio = 0.0226 | std = 0.1600 | días válidos = 1311
price_ema30    : IC medio = 0.0251 | std = 0.1960 | días válidos = 1311


#### 3.5.1. Conclusiones

| Indicador     | IC medio | Desv. Estándar | Comentario                                   |
|---------------|----------|----------------|-----------------------------------------------|
| `price_ema15` | 0.0207   | 0.1361         | Muy débil, con alta variabilidad.             |
| `price_ema20` | 0.0226   | 0.1600         | Ligera mejora, pero aún irrelevante.          |
| `price_ema30` | 0.0251   | 0.1960         | Ligeramente más estable, pero poco útil.      |

<br>

- Todos los indicadores basados en la distancia al EMA presentan **IC medios extremadamente bajos**, lo que indica **muy baja capacidad predictiva** para anticipar el retorno a 30 minutos.

- Aumentar la ventana de la EMA de 15 a 30 no mejora sustancialmente el desempeño; el IC apenas varía y la **desviación estándar aumenta**, lo cual refleja una señal más ruidosa.

- Aunque conceptualmente estos indicadores pueden capturar sobreextensión o reversión, en la práctica **no aportan valor estadístico en este horizonte temporal**.


#### 3.5.2. Recomendaciones

- **No se recomienda incluir `price_ema15`, `price_ema20` ni `price_ema30`** como alpha factors en el modelo actual.

- En su lugar, es preferible utilizar indicadores que midan condiciones extremas desde otros enfoques, como `rsi_14` o `bb_percent_30_20`, que mostraron mucho mejor desempeño.

- Estos indicadores podrían evaluarse nuevamente si se cambia el horizonte de predicción o se combinan con señales de tendencia más fuertes.

### 3.6. **%K Estocástico**

In [33]:
mnq_stoch, stoch_cols = calcular_stochastic()
IC_stoch = calcular_IC(mnq_stoch, 'target_return_30', stoch_cols)

📊 IC promedio por factor (diario) sobre el target_return_30:

stoch_k_14     : IC medio = 0.0610 | std = 0.1992 | días válidos = 1311
stoch_k_20     : IC medio = 0.0629 | std = 0.2075 | días válidos = 1311
stoch_k_30     : IC medio = 0.0679 | std = 0.2237 | días válidos = 1311


#### 3.6.1. Conclusiones

| Indicador     | IC medio | Desv. Estándar | Comentario                                  |
|---------------|----------|----------------|----------------------------------------------|
| `stoch_k_14`  | 0.0610   | 0.1992         | Señal débil y bastante volátil.              |
| `stoch_k_20`  | 0.0629   | 0.2075         | Sin mejora significativa respecto al anterior. |
| `stoch_k_30`  | 0.0679   | 0.2237         | Leve aumento del IC, pero más inestable aún. |

<br>

- Los valores de IC medio para los tres estocásticos son **muy bajos**, lo que sugiere una **capacidad predictiva limitada** sobre el retorno a 30 minutos.

- Aumentar la ventana de cálculo incrementa ligeramente el IC, pero también **aumenta considerablemente la desviación estándar**, lo que indica que la señal es más inconsistente.

- En general, el indicador parece no alinearse bien con la dinámica del target evaluado, posiblemente por su sensibilidad a oscilaciones de corto plazo.

#### 3.6.2. Recomendaciones

- **No se recomienda incluir `stoch_k_14`, `stoch_k_20` ni `stoch_k_30`** como alpha factors principales.

- Dado su bajo IC y alta varianza, estos indicadores pueden introducir **ruido más que señal** en el modelo.

- Podrían reevaluarse en contextos donde se investigue sobrecompra/sobreventa extrema a horizontes más cortos o en combinación con filtros de tendencia.

### 3.7.**%B de Bollinger (Bollinger Band Percent)**


In [34]:
mnq_bollinger, bollinger_cols = calcular_bollinger()
IC_bollinger = calcular_IC(mnq_bollinger, 'target_return_30', bollinger_cols)

📊 IC promedio por factor (diario) sobre el target_return_30:

bb_percent_15_15: IC medio = 0.3986 | std = 0.0628 | días válidos = 1311
bb_percent_20_15: IC medio = 0.4619 | std = 0.0670 | días válidos = 1311
bb_percent_30_15: IC medio = 0.5679 | std = 0.0757 | días válidos = 1311
bb_percent_15_20: IC medio = 0.3986 | std = 0.0628 | días válidos = 1311
bb_percent_20_20: IC medio = 0.4619 | std = 0.0670 | días válidos = 1311
bb_percent_30_20: IC medio = 0.5679 | std = 0.0757 | días válidos = 1311
bb_percent_15_25: IC medio = 0.3986 | std = 0.0628 | días válidos = 1311
bb_percent_20_25: IC medio = 0.4619 | std = 0.0670 | días válidos = 1311
bb_percent_30_25: IC medio = 0.5679 | std = 0.0757 | días válidos = 1311


#### 3.7.1. Conclusiones

| Indicador          | Ventana | Desv. Std | IC medio | Desv. Estándar | Comentario                               |
|--------------------|---------|-----------|----------|----------------|-------------------------------------------|
| `bb_percent_15_15` | 15      | 1.5       | 0.3986   | 0.0628         | Señal útil, aunque más sensible.          |
| `bb_percent_20_15` | 20      | 1.5       | 0.4619   | 0.0670         | Buena señal, más estable.                 |
| `bb_percent_30_15` | 30      | 1.5       | 0.5679   | 0.0757         | Muy fuerte, la mejor entre las variantes. |
| `bb_percent_15_20` | 15      | 2.0       | 0.3986   | 0.0628         | Idéntico a `15_15`, no mejora.            |
| `bb_percent_20_20` | 20      | 2.0       | 0.4619   | 0.0670         | Idéntico a `20_15`, igualmente robusto.   |
| `bb_percent_30_20` | 30      | 2.0       | 0.5679   | 0.0757         | Idéntico a `30_15`, excelente señal.      |
| `bb_percent_15_25` | 15      | 2.5       | 0.3986   | 0.0628         | Sin cambios respecto a otras versiones.   |
| `bb_percent_20_25` | 20      | 2.5       | 0.4619   | 0.0670         | Misma efectividad.                        |
| `bb_percent_30_25` | 30      | 2.5       | 0.5679   | 0.0757         | Consistente como las demás de ventana 30. |

<br>

- Los resultados muestran que **el valor de la desviación estándar (`window_dev`) no afecta el desempeño predictivo**: los valores de IC son idénticos para cada ventana, sin importar si se usa 1.5, 2.0 o 2.5.

- En cambio, **la ventana de cálculo (`window`) sí tiene un impacto claro**:
  - A mayor ventana (de 15 a 30), el IC promedio **aumenta significativamente**.
  - `bb_percent_30_*` alcanza un IC medio de **0.5679**, con desviación estándar moderada, lo que la convierte en una de las señales más fuertes evaluadas.

- Esto sugiere que **condiciones de sobrecompra/sobreventa sostenidas en el tiempo** (no picos breves) están más alineadas con movimientos relevantes a 30 minutos.


#### 3.7.2. Recomendaciones

- **Priorizar `bb_percent_30_20`** (o cualquiera de las variantes con ventana 30), dado que muestran el **mayor IC medio** con buena estabilidad.

- No es necesario incluir todas las combinaciones de `bb_percent`; basta con seleccionar la versión más representativa (`window=30`, cualquier `window_dev`).

- Este indicador debe considerarse como uno de los **factores principales en el modelo predictivo**.

### 3.8. **ATR normalizado (Average True Range / precio)**

In [35]:
mnq_atr, atr_cols = calcular_atr()
IC_atr = calcular_IC(mnq_atr, 'target_return_30', atr_cols)

📊 IC promedio por factor (diario) sobre el target_return_30:

atr_norm       : IC medio = 0.4313 | std = 0.1131 | días válidos = 1311


### 3.9. **ROC (Rate of Change)**

In [36]:
mnq_roc, roc_cols = calcular_roc()
IC_roc = calcular_IC(mnq_roc, 'target_return_30', roc_cols)

📊 IC promedio por factor (diario) sobre el target_return_30:

roc_5          : IC medio = 0.0315 | std = 0.1653 | días válidos = 1311


Evaluación de Indicadores Técnicos usando IC diario
Se utilizó el Information Coefficient (IC) de Spearman para medir la capacidad predictiva de diversos indicadores técnicos sobre el retorno a 30 minutos del índice MNQ. El IC mide la correlación entre el ranking del valor del factor y el ranking del retorno futuro, evaluado día por día.

✅ Factores más prometedores (IC medio alto y estable)
Estos factores presentan un IC medio alto y una desviación estándar baja, lo que indica potencial predictivo consistente:

Factor	IC medio	Std dev	Comentario
rsi_14	0.6387	0.0666	Muy fuerte, consistente y estable. Excelente señal.
bb_percent	0.4619	0.0670	Fuerte y estable. Claramente informativo.
atr_norm	0.4313	0.1131	Buen poder predictivo, aunque más volátil.
rsi_7	0.4740	0.0582	Similar a bb_percent, también robusto.

⚠️ Factores con señal débil o ruidosa
Estos factores tienen un IC medio cercano a cero o alta desviación estándar, por lo que aportan poco valor predictivo y podrían considerarse ruidosos:

Factor	IC medio	Std dev	Comentario
momentum_*	~0.03	0.14–0.20	Casi ruido. No aportan mucho valor.
price_ema20	0.0226	0.1600	Ruido, sin valor informativo significativo.
volume_ratio	0.0048	0.0785	Prácticamente irrelevante en este horizonte.
roc_5	0.0315	0.1653	Similar a momentum_5, bajo poder predictivo.

🧪 Factores mixtos o medianamente útiles
Estos factores presentan una señal moderada con mayor varianza. Podrían combinarse con otros en modelos multivariados:

Factor	IC medio	Std dev	Comentario
macd	0.1962	0.0984	Aporta algo, pero con más ruido.
stoch_k	0.0610	0.1992	Ligera señal, pero con alta varianza.

Criterios de selección
Se definieron los siguientes umbrales para seleccionar los factores más sólidos:

ic_mean ≥ 0.4

ic_std ≤ 0.12

Esto permite incluir factores informativos como atr_norm, sin permitir la entrada de factores inestables o poco útiles.

In [ ]:
def seleccionar_mejores_factores(df_ic: pd.DataFrame,
                                  umbral_ic: float = 0.4,
                                  umbral_std: float = 0.12):
    """
    Filtra factores con IC medio >= umbral_ic y desviación estándar <= umbral_std.

    Args:
        df_ic (pd.DataFrame): DataFrame con columnas ['factor', 'ic_mean', 'ic_std'].
        umbral_ic (float): Mínimo IC medio requerido.
        umbral_std (float): Máxima desviación estándar permitida.

    Returns:
        pd.DataFrame: Subconjunto de factores seleccionados.
    """
    seleccionados = df_ic[(df_ic['ic_mean'] >= umbral_ic) & (df_ic['ic_std'] <= umbral_std)].copy()
    return seleccionados.sort_values(by='ic_mean', ascending=False)

In [ ]:
mejores = seleccionar_mejores_factores(IC_tecnicos)
print(mejores)

📈 Interpretación de los resultados — Information Coefficient (IC)

El **Information Coefficient (IC)** mide la correlación de Spearman entre el valor del factor y el retorno futuro. Es decir, evalúa si el ranking del factor anticipa el ranking del rendimiento posterior.

| 🏅 Top factores con mejor IC | IC Medio | Interpretación práctica                                        |
| ---------------------------- | -------- | -------------------------------------------------------------- |
| **rsi\_14**                  | -0.0534  | Señal clara de reversión. Es el más consistente del conjunto.  |
| **price\_ema20**             | -0.0435  | Reversión hacia la media. Buena señal complementaria.          |
| **bb\_percent**              | -0.0387  | Similar a RSI, sobrecompra/sobreventa. Señal confiable.        |
| **stoch\_k**                 | -0.0374  | Confirma reversión; alternativa válida al RSI.                 |
| **atr\_norm**                | +0.0471  | Único con IC positivo fuerte. Alta volatilidad → continuación. |
| **rsi\_7**                   | -0.0405  | Más sensible que RSI(14), mantiene coherencia.                 |

**Estos pueden capturar efectos de reversión o momentum en condiciones específicas, y probablemente rindan mejor combinados en modelos no lineales (XGBoost, LSTM, etc.).**


📉 Factores con señal débil o nula:

| Factor                       | IC Medio      | Nota                                       |
| ---------------------------- | ------------- | ------------------------------------------ |
| momentum_3/5/10, roc_5   | -0.02 a -0.03 | Señales débiles, sin aporte significativo. |
| macd`                       | -0.0196       | Muy débil para este timeframe.             |
| volume_ratio, body_ratio | ≈ 0           | No aportan valor predictivo.               |



Filtro el dataset para quedarme solo con las factores que aportarían valor a mi modelo:

In [ ]:
# Lista de columnas útiles
keep_cols = [
    'open', 'high', 'low', 'close', 'volume', 'date',
    'rsi_14', 'rsi_7', 'price_ema20', 'bb_percent',
    'stoch_k', 'atr_norm',
    'future_return_5min'
]


# Filtrar el dataset
mnq_intraday_clean = mnq_intraday_factors[keep_cols].dropna()

📌 Notas:

- dropna() elimina las filas con NaN, que suelen ser las primeras del día por el cálculo de ventanas (RSI, ATR, etc.).
- Esto asegura que el dataset esté limpio para entrenamiento de modelos.

In [ ]:
mnq_intraday_clean

##### 2.2. IDEAS PARA FEATURES

🧠 Ideas para combinar:

- Filtrar señales por volatilidad: usar atr_norm como feature auxiliar para condicionar otras señales.
- Ensemble de reversión: combinar rsi, stoch_k, bb_percent en un score compuesto.
- Interacciones: por ejemplo, si rsi está bajo y bb_percent < 0.1 → señal de rebote más fuerte.

###### ✅ 1. reversal_score: Ensemble de reversión suave

Combina señales de sobreventa (RSI, %K, BB) en una puntuación continua. Cuanto más baja, mayor potencial de rebote.

In [ ]:
# Escalar a rango 0-1 donde corresponda
mnq_intraday_clean['rsi_norm'] = mnq_intraday_clean['rsi_14'] / 100
mnq_intraday_clean['stoch_k_norm'] = mnq_intraday_clean['stoch_k'] / 100
# bb_percent ya está entre 0-1

# Reversal Score: promedio de los tres, invertido
mnq_intraday_clean['reversal_score'] = 1 - (
    mnq_intraday_clean['rsi_norm'] +
    mnq_intraday_clean['stoch_k_norm'] +
    mnq_intraday_clean['bb_percent']
) / 3

###### ✅ 2. volatility_filter: Filtro de condiciones de mercado
Crea una feature binaria que indica si estamos en un contexto de alta volatilidad según ATR normalizado.

In [ ]:
mnq_intraday_clean['volatility_filter'] = (mnq_intraday_clean['atr_norm'] > 0.02).astype(int)

El umbral 0.02 es empírico — podés ajustarlo con histogramas o percentiles.

###### ✅ 3. rebound_trigger: Señal fuerte combinada (regla lógica)

Detecta eventos donde:

- RSI está bajo
- %B está por debajo de 0.1 (precio muy fuera de banda)
- Volatilidad es alta

In [ ]:
mnq_intraday_clean['rebound_trigger'] = (
    (mnq_intraday_clean['rsi_14'] < 30) &
    (mnq_intraday_clean['bb_percent'] < 0.1) &
    (mnq_intraday_clean['atr_norm'] > 0.02)
).astype(int)


In [ ]:
mnq_intraday_clean

# 2.3. ANÁLISIS POR CUANTILES

¿Qué hace este código?

Para cada factor:

- Se divide cada día en 5 quintiles (qcut).
- Calcula el retorno medio futuro de cada quintil.
- Acumula los resultados de todos los días válidos.
- Al final, te da el retorno promedio por quintil en todo el dataset.

Esto te permite ver si, por ejemplo, los valores bajos del RSI (Q0) tienden a tener retornos positivos → reversión. O si Q4 tiene retorno positivo → momentum.

In [ ]:
# Factores seleccionados que querés evaluar
factors = ['rsi_14', 'rsi_7', 'price_ema20', 'bb_percent', 'stoch_k', 'atr_norm', 'reversal_score']

target = 'future_return_5min'

# Agrupamos por día
grouped = mnq_intraday_clean.groupby('date')

print("\n📈 Retorno promedio por quintil (promedio diario):\n")

for factor in factors:
    all_quantile_returns = []

    for date, group in grouped:
        df_valid = group[[factor, target]].dropna()

        if len(df_valid) >= 30:
            try:
                df_valid['q'] = pd.qcut(df_valid[factor], 5, labels=False, duplicates='drop')
                mean_ret = df_valid.groupby('q')[target].mean()

                if len(mean_ret) == 5:
                    all_quantile_returns.append(mean_ret.values)
            except ValueError:
                continue  # Si no hay suficientes valores únicos para dividir en quintiles

    if all_quantile_returns:
        avg_qrets = np.mean(all_quantile_returns, axis=0)
        print(f"\n🔹 {factor} – retorno medio por quintil (Q0 a Q4):")
        for i, r in enumerate(avg_qrets):
            print(f"  Q{i}: {r:.5f}")
    else:
        print(f"\n🔹 {factor}: sin suficientes datos diarios para quintiles")

### 📊 Retorno promedio por quintil (Q0 a Q4)


| Factor           | Q0      | Q1      | Q2      | Q3      | Q4       | Interpretación                                               |
|------------------|---------|---------|---------|---------|----------|--------------------------------------------------------------|
| **rsi_14**        | 0.00008 | 0.00001 | 0.00000 | -0.00000| -0.00006 | Reversión clara: Q0 positivo, Q4 negativo                    |
| **rsi_7**         | 0.00006 | 0.00001 | 0.00000 | -0.00000| -0.00005 | Reversión leve, misma estructura que RSI_14                 |
| **price_ema20**   | 0.00008 | 0.00000 | -0.00001| -0.00000| -0.00004 | Sobreextensión a la media sugiere reversión                 |
| **bb_percent**    | 0.00006 | 0.00002 | 0.00000 | -0.00001| -0.00004 | Precio cerca del borde inferior tiende a rebotar            |
| **stoch_k**       | 0.00006 | 0.00002 | 0.00000 | -0.00001| -0.00004 | Señal alternativa de reversión como RSI y BB                |
| **atr_norm**      | -0.00005| -0.00001| 0.00001 | 0.00000 | 0.00008  | Alta volatilidad favorece continuación                      |
| **reversal_score**| -0.00005| -0.00001| -0.00000| 0.00002 | 0.00006  | Score compuesto: buena separación entre extremos            |


✅ Resumen conceptual

Los valores reflejan el retorno promedio futuro a 5 minutos, agrupado por quintiles del factor. Es decir:

- Q0 = valores más bajos del factor (ej: RSI más bajo)
- Q4 = valores más altos del factor

### 📊 Conclusiones clave por tipo de factor


#### 🔄 Factores de reversión (rsi_14, rsi_7, price_ema20, bb_percent, stoch_k)

Patrón observado:
Q0 tiene retorno positivo, Q4 negativo → clara señal de reversión estadística

Por ejemplo: rsi_14

  Q0: +0.00008 → RSI muy bajo → sube
  
  Q4: -0.00006 → RSI muy alto → baja

✅ Interpretación:
Los valores extremos (muy bajos) anticipan rebotes, y los valores altos anticipan caídas, aunque el efecto sea leve.

#### 🔊 Factor de volatilidad (atr_norm)


Patrón:
Q4 tiene retorno positivo → alta volatilidad favorece continuación

Ejemplo:

atr_norm

  Q0: -0.00005 → baja volatilidad → mercado plano

  Q4: +0.00008 → alta volatilidad → rompe en una dirección


✅ Interpretación:
Este factor no predice dirección por sí solo, pero ayuda a filtrar cuándo las señales funcionan mejor (alta volatilidad = movimiento significativo).

#### 🧠 Feature combinada reversal_score


Patrón observado: Q0 negativo, Q4 positivo → signo invertido por construcción

Y se comporta bien: Q4 = +0.00006

✅ Interpretación:

Confirma que el score compuesto refleja la lógica de reversión. Es más fácil de interpretar para un modelo ML y combina múltiples señales.

### 🎯 Recomendación general

Los factores muestran consistencia en su estructura predictiva: reversión suave, efecto de volatilidad, combinación útil.

Aunque los valores absolutos de retorno son pequeños (esperado en timeframe de 1 minuto), la dirección del efecto está clara y sistemática, lo cual es valioso para modelos de ML.

Estas señales pueden ser muy efectivas en modelos que aprovechan la combinación y no linealidad (ej. XGBoost, LSTM).

## ESTABILIDAD DE ALPHA FACTORS?


In [ ]:
mnq_intraday_clean

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

# Suponemos que ya cargaste mnq_intraday_clean con estos factores
alpha_factors = ['rsi_14', 'rsi_7', 'price_ema20', 'bb_percent', 'stoch_k', 'atr_norm', 'reversal_score']
target = 'future_return_5min'

# Agrupar por día
grouped = mnq_intraday_clean.groupby('date')

# Calcular IC diario
ic_diarios = {factor: [] for factor in alpha_factors}
fechas = []

for date, group in grouped:
    fechas.append(date)
    for factor in alpha_factors:
        df_valid = group[[factor, target]].dropna()
        if len(df_valid) >= 30:
            ic = spearmanr(df_valid[factor], df_valid[target]).correlation
        else:
            ic = np.nan
        ic_diarios[factor].append(ic)

# Convertir a DataFrame de IC diarios
df_ic = pd.DataFrame(ic_diarios, index=pd.to_datetime(fechas))

# Graficar la media móvil de 20 días
plt.figure(figsize=(14, 6))
for factor in alpha_factors:
    df_ic[factor].rolling(20).mean().plot(label=factor)

plt.title("📈 IC promedio móvil (20 días) por alpha factor")
plt.xlabel("Fecha")
plt.ylabel("Information Coefficient (Spearman)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()